# reaction-network (Demo Notebook): Networks

## Author: Matthew McDermott
## Last Updated: 11/16/21

**If you use this code in your work, please consider citing the following paper:**

McDermott, M. J., Dwaraknath, S. S., and Persson, K. A. (2021). A graph-based network for predicting chemical reaction pathways in solid-state materials synthesis. 
Nature Communications, 12(1). https://doi.org/10.1038/s41467-021-23339-x

In [23]:
import logging 

from pymatgen.ext.matproj import MPRester
from rxn_network.enumerators.basic import BasicEnumerator, BasicOpenEnumerator
from rxn_network.enumerators.minimize import MinimizeGibbsEnumerator, MinimizeGrandPotentialEnumerator
from pymatgen.analysis.phase_diagram import PhaseDiagram, PDPlotter
from rxn_network.thermo.chempot_diagram import ChemicalPotentialDiagram

from rxn_network.costs.softplus import Softplus
from pymatgen.core.composition import Composition, Element
from rxn_network.entries.entry_set import GibbsEntrySet
from rxn_network.network.network import ReactionNetwork
from rxn_network.entries.nist import NISTReferenceEntry
from rxn_network.reactions.computed import ComputedReaction
from rxn_network.reactions.reaction_set import ReactionSet
from rxn_network.reactions.open import OpenComputedReaction
from rxn_network.network.entry import NetworkEntry, NetworkEntryType
from rxn_network.network.visualize import plot_network_on_graphistry, plot_network
from rxn_network.pathways.solver import PathwaySolver

import graphistry
import graph_tool.all as gt

logging.info("Logging initialized")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with MPRester() as mpr:  # Insert your Materials Project API key here (if needed)
    entries = mpr.get_entries_in_chemsys("Y-Mn-O-Li-Cl-C", inc_structure="final")

This database version has changed from the database last accessed (2020_09_08).
Please see release notes on materialsproject.org for information about what has changed.


  0%|          | 0/898 [00:00<?, ?it/s]

In [4]:
entry_set = GibbsEntrySet.from_entries(entries, 900, include_barin_data=False).filter_by_stability(0.0)

In [5]:
be = BasicEnumerator()
cf = Softplus(900)

In [6]:
rn = ReactionNetwork(entry_set, [BasicEnumerator()], cf)

In [7]:
rn.build()

BasicEnumerator:   0%|          | 0/57 [00:00<?, ?it/s]

In [8]:
rn.set_precursors([entry_set.get_min_entry_by_formula("Li2CO3"), 
                   entry_set.get_min_entry_by_formula("Mn2O3"), 
                   entry_set.get_min_entry_by_formula("YCl3")])

In [9]:
from rxn_network.network.entry import NetworkEntryType

In [10]:
NetworkEntryType.Target

<NetworkEntryType.Target: 4>

In [11]:
for i in rn.graph.vertices():
    if rn.graph.vp["type"][i] == 4:
        print(i)

In [17]:
rn.set_target("YMnO3")

In [18]:
paths = rn.find_pathways(["YMnO3", "LiCl", "CO2"], k=5)

PATHS to YMnO3 

--------------------------------------- 

Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
YCl3 + 1.5 Li2O -> 0.5 Y2O3 + 3 LiCl (dG = -0.297 eV/atom) 
Y2O3 + Mn2O3 -> 2 YMnO3 (dG = -0.069 eV/atom) 
Total Cost: 0.789 

6 Mn2O3 -> O2 + 4 Mn3O4 (dG = 0.043 eV/atom) 
O2 + 2 YCl3 -> 2 YClO + 2 Cl2 (dG = 0.011 eV/atom) 
YClO + 0.3333 Mn2O3 -> 0.3333 YCl3 + 0.6667 YMnO3 (dG = 0.0 eV/atom) 
Total Cost: 0.808 

6 Mn2O3 -> O2 + 4 Mn3O4 (dG = 0.043 eV/atom) 
O2 + 1.333 YCl3 -> 2 Cl2 + 0.6667 Y2O3 (dG = 0.078 eV/atom) 
Y2O3 + Mn2O3 -> 2 YMnO3 (dG = -0.069 eV/atom) 
Total Cost: 0.808 

Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
YCl3 + Li2O -> YClO + 2 LiCl (dG = -0.273 eV/atom) 
YClO + 0.3333 Mn2O3 -> 0.3333 YCl3 + 0.6667 YMnO3 (dG = 0.0 eV/atom) 
Total Cost: 0.809 

Mn2O3 -> MnO + MnO2 (dG = 0.097 eV/atom) 
YCl3 + 1.5 MnO -> 0.5 Y2O3 + 1.5 MnCl2 (dG = 0.051 eV/atom) 
Y2O3 + Mn2O3 -> 2 YMnO3 (dG = -0.069 eV/atom) 
Total Cost: 0.815 

PATHS to LiCl 

--------------------------------------

In [19]:
ps = PathwaySolver(rn.entries, paths, Softplus(900)) # open_elem="O", chempot=0

In [20]:
product_entries = []
for i in ["YMnO3","LiCl","CO2"]:
    product_entries.append(entry_set.get_min_entry_by_formula(i))
    
net_rxn = ComputedReaction.balance(rn.precursors,product_entries)
net_rxn

YCl3 + 0.5 Mn2O3 + 1.5 Li2CO3 -> YMnO3 + 3 LiCl + 1.5 CO2

In [21]:
a = ps.solve(net_rxn, max_num_combos=5, intermediate_rxn_energy_cutoff=0.0, filter_interdependent=True)

BasicEnumerator:   0%|          | 0/36 [00:00<?, ?it/s]

PathwaySolver:   0%|          | 0/1 [00:00<?, ?it/s]

PathwaySolver:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
for path in a:
    print(path)
    print("\n")

Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Li2O + YCl3 -> 2 LiCl + YClO (dG = -0.273 eV/atom) 
Li2CO3 + Mn2O3 -> CO2 + 2 LiMnO2 (dG = 0.08 eV/atom) 
LiMnO2 + YClO -> LiCl + YMnO3 (dG = -0.081 eV/atom) 
Average Cost: 0.267


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.069 eV/atom) 
Li2O + YCl3 -> 2 LiCl + YClO (dG = -0.273 eV/atom) 
0.5 Li2O + YClO -> LiCl + 0.5 Y2O3 (dG = -0.136 eV/atom) 
Average Cost: 0.268


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
1.5 Li2O + YCl3 -> 3 LiCl + 0.5 Y2O3 (dG = -0.297 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.069 eV/atom) 
Li2O + YCl3 -> 2 LiCl + YClO (dG = -0.273 eV/atom) 
0.5 Li2O + YClO -> LiCl + 0.5 Y2O3 (dG = -0.136 eV/atom) 
Average Cost: 0.274


Li2CO3 -> CO2 + Li2O (dG = 0.277 eV/atom) 
1.5 Li2O + YCl3 -> 3 LiCl + 0.5 Y2O3 (dG = -0.297 eV/atom) 
Mn2O3 + Y2O3 -> 2 YMnO3 (dG = -0.069 eV/atom) 
Average Cost: 0.278


